<a href="https://colab.research.google.com/github/YHL04/tryingoutideas/blob/main/longformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch


def _skew(x, direction, padding_value):
  '''Convert diagonals into columns (or columns into diagonals depending on `direction`'''
  x_padded = F.pad(x, direction, value=padding_value)
  x_padded = x_padded.view(*x_padded.size()[:-2], x_padded.size(-1), x_padded.size(-2))
  return x_padded


def _chunk(x, w):
  '''convert into overlapping chunkings. Chunk size = 2w, overlap size = w'''

  # non-overlapping chunks of size = 2w
  x = x.view(x.size(0), x.size(1) // (w * 2), w * 2, x.size(2))

  # use `as_strided` to make the chunks overlap with an overlap size = w
  chunk_size = list(x.size())
  chunk_size[1] = chunk_size[1] * 2 - 1

  chunk_stride = list(x.stride())
  chunk_stride[1] = chunk_stride[1] // 2

  return x.as_strided(size=chunk_size, stride=chunk_stride)


def sliding_chunks_matmul_qk(q, k, w):
  bsz, seqlen, num_heads, head_dim = q.size()

  chunks_count = seqlen // w - 1

  q = q.transpose(1, 2).reshape(bsz * num_heads, seqlen, head_dim)
  k = k.transpose(1, 2).reshape(bsz * num_heads, seqlen, head_dim)

  chunk_q = _chunk(q, w)
  chunk_k = _chunk(k, w)

  chunk_attn = torch.einsum('bcxd,bcyd->bcxy', (chunk_q, chunk_k))  # multiply



In [ ]:

x = torch.arange(8).reshape(2, 4)
y = torch.arange(12).reshape(3, 4) + 1

print(x)
print(y)


print(torch.matmul(x, y.transpose(0, 1)))
print(torch.einsum('xd,yd->xy', (x, y)))




tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])
tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])
tensor([[ 20,  44,  68],
        [ 60, 148, 236]])
tensor([[ 20,  44,  68],
        [ 60, 148, 236]])


In [ ]:

x = torch.arange(16).reshape(4, 4)
padded = torch.nn.functional.pad(x, (0, 0, 0, 1))
print(x)
print(padded)
print(padded.view(padded.size(-1), padded.size(-2)))


tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [ 0,  0,  0,  0]])
tensor([[ 0,  1,  2,  3,  4],
        [ 5,  6,  7,  8,  9],
        [10, 11, 12, 13, 14],
        [15,  0,  0,  0,  0]])
